In [24]:
!pip install pytorch_tabnet
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [25]:
X_full = pd.read_csv('/workspaces/2025SE-Mike.N-HSC-AT2/data/concrete_preprocessed.csv')
y = np.log1p(X_full['strength'])
X = X_full.drop(['strength'], axis=1)

In [26]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 965 entries, 0 to 964
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PC1     965 non-null    float64
 1   PC2     965 non-null    float64
 2   PC3     965 non-null    float64
 3   PC4     965 non-null    float64
 4   PC5     965 non-null    float64
 5   PC6     965 non-null    float64
 6   PC7     965 non-null    float64
 7   PC8     965 non-null    float64
 8   PC9     965 non-null    float64
 9   PC10    965 non-null    float64
 10  PC11    965 non-null    float64
dtypes: float64(11)
memory usage: 83.1 KB


In [27]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_valid, y_train_valid, test_size=0.2, random_state=42)

In [28]:
X_train_valid      = X_train_valid.to_numpy()
y_train_valid      = y_train_valid.to_numpy().reshape(-1, 1)
X_test = X_test.to_numpy()

In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in kf.split(X_train_valid):
    X_train, X_valid = X_train_valid[train_index], X_train_valid[test_index]
    y_train, y_valid = y_train_valid[train_index], y_train_valid[test_index]
    regressor = TabNetRegressor(verbose=0,seed=42)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=500, max_epochs=2500,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(np.expm1(regressor.predict(X_test)))

predictions = np.mean(predictions_array,axis=0)


Early stopping occurred at epoch 500 with best_epoch = 0 and best_val_0_rmse = 3.87935


/usr/local/python/3.12.1/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 500 with best_epoch = 0 and best_val_0_rmse = 3.95946


/usr/local/python/3.12.1/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 500 with best_epoch = 0 and best_val_0_rmse = 3.79565


/usr/local/python/3.12.1/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
print("The CV score is %.5f" % np.mean(CV_score_array,axis=0))

The CV score is 3.87703


In [ ]:
RMSLE = np.sqrt(root_mean_squared_error(y_test, predictions))
print("The LB score is %.5f" % RMSLE )

The LB score is 1.89970
